### Création d’un index simple

- Si on effectue souvent des recherches par email, on peut ajouter un index sur cette colonne :

```sql
CREATE INDEX idx_utilisateurs_email ON utilisateurs(email);
```

#### Test de performance avec EXPLAIN ANALYZE

```sql
EXPLAIN ANALYZE SELECT * FROM utilisateurs WHERE email = 'bob@example.com';
```

Filter: ((email)::text = 'bob@example.com'::text)  
Rows Removed by Filter: 1  
Planning Time: 0.272 ms  
Execution Time: 0.026 ms  

### Index sur plusieurs colonnes

- Si une requête filtre souvent sur ```nom``` et ```email```, on peut créer un index composite :

```sql
CREATE INDEX idx_utilisateurs_nom_email ON utilisateurs(nom, email);
```

Attention : Un index composite n’est utile que si la requête filtre sur la première colonne (ici nom), sinon il ne sera pas utilisé.

### Index GIN pour les recherches textuelles avancées

- Pour rechercher des mots dans un texte, PostgreSQL propose l’index ```GIN``` :

```sql
CREATE INDEX idx_utilisateurs_nom_gin ON utilisateurs USING gin(to_tsvector('french', nom));
```

- Puis, une recherche optimisée :

```sql
SELECT * FROM utilisateurs WHERE to_tsvector('french', nom) @@ to_tsquery('Martin');
```

### Transactions et performances

Les transactions garantissent que plusieurs opérations s’exécutent ensemble ou pas du tout (Atomicité).

```sql
BEGIN;
UPDATE utilisateurs SET age = age + 1 WHERE id = 1;
INSERT INTO commandes (utilisateur_id, produit, prix) VALUES (1, 'Clavier', 49.99);
COMMIT;
```

Si une erreur survient, on peut annuler avec :

```sql
ROLLBACK;
```

### Optimisation avec ```BATCH INSERT```

Au lieu d’insérer ligne par ligne (lent) :

```sql
INSERT INTO commandes (utilisateur_id, produit, prix) 
VALUES (1, 'Clavier', 49.99);
```


Il vaut mieux insérer plusieurs lignes en une seule requête :

```sql
INSERT INTO commandes (utilisateur_id, produit, prix) 
VALUES (1, 'Clavier', 49.99), (2, 'Souris', 29.99), (3, 'Écran', 199.99);
```

= Gains de performance 🚀 : Moins d’allers-retours entre le client et le serveur.

### Eviter les SELECT *

✅ Bonne pratique :

```sql
SELECT nom, email FROM utilisateurs;
```

### Choisir le bon type de données

- ```VARCHAR(255)``` ❌ (inutilement grand)
- ```VARCHAR(50)``` ✅ (plus rapide en RAM)
- ```TEXT``` ✅ (si taille variable et texte long)
- ```BOOLEAN``` ✅ (plutôt que ```INTEGER pour stocker ```0/1```)
- ```TIMESTAMP WITH TIME ZONE``` ✅ (évite les problèmes de fuseaux horaires)

### Partitionnement des tables

- Si une table devient très grande (millions de lignes), on peut la partitionner :

```sql
CREATE TABLE commandes_2024 PARTITION OF commandes 
FOR VALUES FROM ('2024-01-01') TO ('2024-12-31');
```

= Chaque année aura sa propre partition, améliorant les performances.